## Load training transaction data

In [31]:
import pandas as pd

# Loading the data
df_tr = pd.read_csv("data/fraudTrain.csv")
pd.set_option('display.max_columns', None)
df_tr.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [32]:
# Drop the column named 'Unnamed: 0' (unnecessary index column)
df_tr = df_tr.drop(columns=['Unnamed: 0'])

# Print the total number of transactions in the training dataset
print("Number of transactions in the training dataset : ", len(df_tr))

# Identify duplicated rows in the dataset 
duplicate_rows = df_tr[df_tr.duplicated()]
print("Number of duplicated transactions :", len(duplicate_rows))

Number of transactions in the training dataset :  1296675
Number of duplicated transactions : 0


In [51]:
# Detect missing values
missing_counts = df_tr.isna().sum()
missing_percent = (missing_counts / len(df_tr)) * 100

# Combine into one table
missing_summary = pd.DataFrame({
    'missing_count': missing_counts,
    'missing_percent': missing_percent
}).sort_values(by='missing_percent', ascending=False)

print(missing_summary)

                       missing_count  missing_percent
trans_date_trans_time              0              0.0
cc_num                             0              0.0
merchant                           0              0.0
category                           0              0.0
amt                                0              0.0
first                              0              0.0
last                               0              0.0
gender                             0              0.0
street                             0              0.0
city                               0              0.0
state                              0              0.0
zip                                0              0.0
lat                                0              0.0
long                               0              0.0
city_pop                           0              0.0
job                                0              0.0
dob                                0              0.0
trans_num                   

In [52]:
# Automatically convert the columns of df_tr to the best possible data types
df_tr = df_tr.convert_dtypes()

# Display the data types of all columns after conversion
df_tr.dtypes

trans_date_trans_time    datetime64[ns]
cc_num                   string[python]
merchant                       category
category                       category
amt                             Float64
first                          category
last                           category
gender                         category
street                         category
city                           category
state                          category
zip                            category
lat                             Float64
long                            Float64
city_pop                          Int64
job                            category
dob                      datetime64[ns]
trans_num                string[python]
unix_time                         Int64
merch_lat                       Float64
merch_long                      Float64
is_fraud                          Int64
dtype: object

In [34]:
# Convert date/time columns
df_tr['trans_date_trans_time'] = pd.to_datetime(df_tr['trans_date_trans_time'], errors='coerce')
df_tr['dob'] = pd.to_datetime(df_tr['dob'], errors='coerce')

# Keep IDs as string/object
df_tr['cc_num'] = df_tr['cc_num'].astype(str)
df_tr['trans_num'] = df_tr['trans_num'].astype(str)

# Convert categorical/text columns
categorical_cols = ['merchant', 'category', 'first', 'last', 'gender', 
                    'street', 'city', 'state', 'zip', 'job']
for col in categorical_cols:
    df_tr[col] = df_tr[col].astype('category')

# Check final dtypes
print(df_tr.dtypes)

trans_date_trans_time    datetime64[ns]
cc_num                           object
merchant                       category
category                       category
amt                             Float64
first                          category
last                           category
gender                         category
street                         category
city                           category
state                          category
zip                            category
lat                             Float64
long                            Float64
city_pop                          Int64
job                            category
dob                      datetime64[ns]
trans_num                        object
unix_time                         Int64
merch_lat                       Float64
merch_long                      Float64
is_fraud                          Int64
dtype: object


In [53]:
# Count the occurrences of each class
# The 'normalize=True' parameter converts counts into proportions
df_tr['is_fraud'].value_counts(normalize=True)


is_fraud
0    0.994211
1    0.005789
Name: proportion, dtype: Float64

## Check uniqueness of cc_num per person

#### Si cc_num correspond réellement à une carte unique, alors toutes les transactions ayant la même cc_num devraient avoir les mêmes valeurs de first, last, gender, dob et job.
#### Hypothèse : cc_num correspond réellement à une carte unique.

In [63]:
# Count how many distinct (first, last, gender, dob, job) combinations exist for each cc_num
combo_counts = (
    df_tr.groupby("cc_num")[["first", "last", "gender", "dob", "job"]]
      .nunique()
)

# Check if (first, last, gender, dob, job) are always single-valued per cc_num
violations = combo_counts[
    (combo_counts["first"] > 1) |
    (combo_counts["last"] > 1) |
    (combo_counts["gender"] > 1) |
    (combo_counts["dob"] > 1) |
    (combo_counts["job"] > 1)
]

if violations.empty:
    print("Each cc_num value corresponds to a unique combination of (first, last, gender, dob, job).")
else:
    print("Some cc_num values map to multiple (first, last, gender, dob, job) combinations:")
    print(violations)


Each cc_num value corresponds to a unique combination of (first, last, gender, dob, job).


In [61]:
# Count how many distinct (street, city, state, zip, lat, long, city_pop) combinations exist for each cc_num
combo_counts = (
    df_tr.groupby("cc_num")[["street", "city", "state", "zip", "lat", "long", "city_pop"]]
      .nunique()
)

# Check if address columns are always single-valued per cc_num
violations = combo_counts[
    (combo_counts["street"] > 1) | 
    (combo_counts["city"] > 1) | 
    (combo_counts["state"] > 1) | 
    (combo_counts["zip"] > 1) | 
    (combo_counts["lat"] > 1) | 
    (combo_counts["long"] > 1) | 
    (combo_counts["city_pop"] > 1)
]

if violations.empty:
    print("Each cc_num value corresponds to a unique combination of (street, city, state, zip, lat, long, city_pop).")
else:
    print(f"Found {len(violations)} cc_num values that map to multiple address combinations:")
    print(violations)
    

Each cc_num value corresponds to a unique combination of (street, city, state, zip, lat, long, city_pop).


# Uniqueness of merchant properties

In [ ]:
# Check if each merchant has a unique (category, merch_lat, merch_long) combination
combo_counts = (
    df_tr.groupby("merchant")[["category", "merch_lat", "merch_long"]]
      .nunique()
)

# Check if merchant attributes are always single-valued
violations = combo_counts[
    (combo_counts["category"] > 1) | 
    (combo_counts["merch_lat"] > 1) | 
    (combo_counts["merch_long"] > 1)
]

if violations.empty:
    print("Each merchant value corresponds to a unique combination of (category, merch_lat, merch_long).")
elif (combo_counts["category"] == 1).all():
    print("Category is unique by merchant but (merch_lat, merch_long) are unique by transaction")
else:
    print(f"Found {len(violations)} merchants that map to multiple (category, merch_lat, merch_long) combinations:")
    print(violations)    

Found 693 merchants that map to multiple (category, merch_lat, merch_long) combinations:
                                  category  merch_lat  merch_long
merchant                                                         
fraud_Abbott-Rogahn                      1       1844        1844
fraud_Abbott-Steuber                     1       1763        1763
fraud_Abernathy and Sons                 1       1751        1751
fraud_Abshire PLC                        1       1895        1895
fraud_Adams, Kovacek and Kuhlman         1        940         940
...                                    ...        ...         ...
fraud_Zemlak Group                       1       1297        1297
fraud_Zemlak, Tillman and Cremin         1       1803        1804
fraud_Ziemann-Waters                     1       1695        1695
fraud_Zieme, Bode and Dooley             1       2560        2560
fraud_Zulauf LLC                         1       1783        1783

[693 rows x 3 columns]


/tmp/ipykernel_166772/2413910178.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_tr.groupby("merchant")[["category", "merch_lat", "merch_long"]]


In [ ]:
print("Number of distinct categories : ", df_tr["category"].nunique())

# Get merchants that have more than one unique category
merchant_categories = df_tr.groupby("merchant")["category"].nunique()

# Filter merchants with more than one category
multi_category_merchants = merchant_categories[merchant_categories > 1]

print(f"Found {len(multi_category_merchants)} merchants with multiple categories:")

# Show the actual categories for each merchant
for merchant in multi_category_merchants.index:
    categories = df_tr[df_tr["merchant"] == merchant]["category"].unique()
    print(f"{merchant}: {list(categories)}")

Number of distinct categories :  14
Found 7 merchants with multiple categories:
fraud_Boyer PLC: ['misc_pos', 'shopping_net']
fraud_Cormier LLC: ['shopping_net', 'health_fitness']
fraud_Dickinson Ltd: ['misc_pos', 'personal_care']
fraud_Gislason Group: ['misc_pos', 'travel']
fraud_Kilback LLC: ['grocery_pos', 'food_dining']
fraud_Kuhn LLC: ['misc_net', 'shopping_pos']
fraud_Schumm PLC: ['shopping_net', 'health_fitness']


/tmp/ipykernel_166772/3933314602.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merchant_categories = df_tr.groupby("merchant")["category"].nunique()


# What is unix_time ?

In [82]:
# Convert columns to datetime
df_tr['trans_date_trans_time'] = pd.to_datetime(df_tr['trans_date_trans_time'])
df_tr['unix_datetime'] = pd.to_datetime(df_tr['unix_time'], unit='s')

# Compute difference in days
diff_days = (df_tr['trans_date_trans_time'] - df_tr['unix_datetime']).dt.days

# Check if all rows have the same difference
all_same = diff_days.nunique() == 1
if all_same:
    print(f"All rows have the same difference of {diff_days.iloc[0]} days.")
else:
    print("Distinct differences (in days) between trans_date_trans_time and unix_time: ", diff_days.unique())
    if set(diff_days.unique()) == {2557, 2556}:
        print("Most rows have a difference of 2557 days (≈7 years including leap years)")

Distinct differences (in days) between trans_date_trans_time and unix_time:  [2557 2556]
Most rows have a difference of 2557 days (≈7 years including leap years)


## Detect cc_num with multiple transactions at the same time

#### Une carte bancaire réelle ne peut normalement pas effectuer plusieurs transactions strictement au même instant (même seconde).
#### Hypothèse : cc_num représente un identifiant quasi-unique d'une carte réelle.

In [43]:
# Group by cc_num and trans_date_trans_time and count distinct trans_num
counts_df = (
    df_tr.groupby(["cc_num", "trans_date_trans_time"])['trans_num']
      .nunique()
      .to_frame('count')
)

# Keep only (cc_num, trans_date_trans_time) paris with distinct trans_num > 1
duplicates_same_second = counts_df[counts_df["count"] > 1]

if duplicates_same_second.empty:
    print("No cc_num has more than one distinct trans_num at the same second.")
else:
    print("Some cc_num values have multiple distinct trans_num within the same second:")
    print(duplicates_same_second.sort_values("count", ascending=False))
    print("Number of cc_num with more than one distinct trans_num at the same second:", len(duplicates_same_second))


Some cc_num values have multiple distinct trans_num within the same second:
                                           count
cc_num              trans_date_trans_time       
180036456789979     2019-08-11 19:26:21        2
2383461948823908    2019-12-06 08:43:10        2
3506042666828517    2019-04-18 17:32:31        2
3517527805128735    2020-03-14 02:15:53        2
3533012926413100    2020-05-21 15:52:29        2
3553629419254918    2019-03-10 02:31:27        2
3560318482131952    2019-10-31 01:16:14        2
3595192916105588    2019-01-16 05:51:27        2
374930071163758     2019-06-16 23:58:35        2
375082648741747     2019-06-28 21:25:50        2
4715741951931168360 2020-03-24 17:28:55        2
4736845434667908128 2019-12-25 00:01:08        2
4904681492230012    2020-03-15 00:34:58        2
4933461930348832    2019-04-14 16:33:06        2
581686439828        2019-08-26 21:47:51        2
6011504998544485    2019-09-23 16:43:49        2
6011652924285713    2020-06-02 20:14:33   